#Asignatura: Visión por Computador

##Alumnos: Yeray Álvarez-Buylla Parra, María Elena Navarro Santana

##Práctica 5

###Paquetes necesarios

In [13]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image

In [17]:
# Máscara en mediapipe
import cv2
import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame = cv2.flip(frame,1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)
        if results.multi_face_landmarks is not None:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(frame, face_landmarks,
                    mp_face_mesh.FACEMESH_CONTOURS,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1),
                    mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=1))
        cv2.imshow("Frame", frame)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [20]:
# Carga la textura para los labios
lip_texture = cv2.imread('C:/Users/lenin/Pictures/labios_negro.jpg')
if lip_texture is None:
    print("Error: No se pudo cargar la textura de los labios. Verifica la ruta del archivo.")
    exit()
lip_red_texture = cv2.imread('C:/Users/lenin/Pictures/pintalabios_textura.jpg')
if lip_texture is None:
    print("Error: No se pudo cargar la textura de los labios. Verifica la ruta del archivo.")
    exit()
# Carga la imagen de cabello usando PIL para manejar el canal alfa
try:
    hair_image_pil = Image.open('C:/Users/lenin/Documents/Universidad_2024-2025/VC/Práctica_5/P5/Filtros/straight_hair.png')
    hair_image = np.array(hair_image_pil)
    hair_image_pil_red = Image.open('C:/Users/lenin/Documents/Universidad_2024-2025/VC/Práctica_5/P5/Filtros/red_hair.png')
    
    hair_image_red = np.array(hair_image_pil_red)
    print("Imagen de cabello cargada correctamente. Dimensiones:", hair_image.shape)
except Exception as e:
    print("Error: No se pudo cargar la imagen de cabello con Pillow. Detalles:", e)
    exit()

mp_face_mesh = mp.solutions.face_mesh
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Índices de los puntos exteriores de los labios y límites de la cara
OUTER_LIPS = [78,191,80,81,82,13,312,311,310,415,308,324,318,402,317,14,87,178,88,95,185,40,39,37,0,267,269,270,409,291,375,321,405,314,17,84,181,91,146]
FOREHEAD = 10
CHIN = 152
LEFT_FACE = 234
RIGHT_FACE = 454

scale_factor = 3.0
filter_active = False  # Para activar o desactivar el filtro

with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while True:
        ret, frame = cap.read()
       
        if not ret:
            break
        
        frame_rgb = frame
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            frame_gray = cv2.cvtColor(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)

            if filter_active:
                    frame_filter = frame_gray
                    lip=lip_texture
                    hair=hair_image_pil
                    shape_factor=(2.25,3.25)
                    position_offset=(0.2,2)
            else:
                    frame_filter = frame
                    lip=lip_red_texture
                    hair=hair_image_pil_red
                    shape_factor=(2.4,3)
                    position_offset=(0.3,2)

            for face_landmarks in results.multi_face_landmarks:
                # *** Superponer labios con textura negra ***
                outer_lips_points = [(int(face_landmarks.landmark[i].x * frame.shape[1]),
                                      int(face_landmarks.landmark[i].y * frame.shape[0])) for i in OUTER_LIPS]
                
                mask = np.zeros_like(frame, dtype=np.uint8)
                outer_lips_points_array = np.array([outer_lips_points], np.int32)
                cv2.fillPoly(mask, [outer_lips_points_array], (255, 255, 255))
                
                
                resized_texture = cv2.resize(lip, (frame_filter.shape[1], frame_filter.shape[0]))
                
                lip_area = cv2.bitwise_and(resized_texture, resized_texture, mask=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))
                background = cv2.bitwise_and(frame_filter, frame_filter, mask=cv2.bitwise_not(cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)))
                frame = cv2.add(background, lip_area)
                # *** Superponer cabello ***
                forehead_point = face_landmarks.landmark[FOREHEAD]
                chin_point = face_landmarks.landmark[CHIN]
                left_face_point = face_landmarks.landmark[LEFT_FACE]
                right_face_point = face_landmarks.landmark[RIGHT_FACE]

                forehead_y = int(forehead_point.y * frame.shape[0])
                chin_y = int(chin_point.y * frame.shape[0])
                left_x = int(left_face_point.x * frame.shape[1])
                right_x = int(right_face_point.x * frame.shape[1])

                hair_height = int((chin_y - forehead_y) * shape_factor[0]) 
                hair_width = int((right_x - left_x) * shape_factor[1]) 

                resized_hair_pil = hair.resize((hair_width, hair_height), Image.LANCZOS)
                resized_hair = np.array(resized_hair_pil)
                
                hair_bgr = resized_hair[:, :, :3]
                hair_bgr =  cv2.cvtColor(hair_bgr, cv2.COLOR_RGB2BGR) 
                hair_alpha = resized_hair[:, :, 3] / 255.0
                
                y_offset = max(0, forehead_y - int(hair_height * position_offset[0]))
                x_offset = left_x - int((hair_width - (right_x - left_x)) / position_offset[1])

                y1, y2 = y_offset, y_offset + hair_height
                x1, x2 = x_offset, x_offset + hair_width
                if y2 > frame.shape[0] or x2 > frame.shape[1] or y1 < 0 or x1 < 0:
                    continue
          
                roi = frame[y1:y2, x1:x2]

                for c in range(3):
                    roi[:, :, c] = roi[:, :, c] * (1 - hair_alpha) + hair_bgr[:, :, c] * hair_alpha

                frame[y1:y2, x1:x2] = roi

                # Muestra la cámara en color sin el filtro cuando filter_active es False
                cv2.imshow("Lipstick and Hair Overlay Filter", frame)

        k = cv2.waitKey(1) & 0xFF
        if k == 27:  # ESC para salir
            break
        elif k == ord('1'):  # Tecla "1" para activar/desactivar el filtro
            filter_active = not filter_active

cap.release()
cv2.destroyAllWindows()

Imagen de cabello cargada correctamente. Dimensiones: (505, 494, 4)
